In [1]:
import modal

assert modal.__version__ > "0.49.0"
modal.__version__

'0.73.7'

In [2]:
from modal import app

app = modal.App(name="example-basic-notebook-app")

In [3]:
def double(x: int) -> int:
    return x + x


double(5)

10

In [15]:
image = (
    modal.Image.debian_slim(python_version="3.11")
    .apt_install(
        "git",
        "libgl1",
        "libglu1-mesa",
        "freeglut3-dev",
        "libxrender1",
        "gmsh",
        "python3-gmsh",
        "xvfb",
        "curl"
    )
    .pip_install("uv")
    .run_commands("uv pip install --system --compile-bytecode trimesh==4.5.3 jupyterlab")
    .run_commands("uv pip install --system --compile-bytecode git+https://github.com/CadQuery/cadquery.git@e99a15df3cf6a88b69101c405326305b5db8ed94")
)

In [9]:
image_test = modal.Image.debian_slim(python_version="3.10").pip_install("jupyterlab")

In [16]:
@app.function()
def double_with_modal(x: int) -> int:
    return x + x


@app.function(cpu=10, image=image)
def quadruple(x: int) -> int:
    if x <= 1_000_000:
        return double(x) + double(x)
    else:
        return double_with_modal.remote(x) + double_with_modal.remote(x)


with app.run():
    print(quadruple.local(100))   # running locally
    print(quadruple.remote(100))  # run remotely

400
400
